In [1]:
exec(open("./funcs/tool_funcs.py").read())


In [2]:
def dijkstra_show_path_minimax_path(adj_matrix, start, end):
 
    n = len(adj_matrix)
    distances = [float('inf')] * n
    previous_nodes = [-1] * n
    distances[start] = 0
    priority_queue = [(0, start)]  # (distance, node)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Stop if we reached the destination node
        if current_node == end:
            break

        # If the distance is no longer optimal, skip
        if current_distance > distances[current_node]:
            continue

        # Explore neighbors
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight > 0:  # Only consider edges with weight > 0
                distance = max(current_distance, weight)

                # Update distance if it's better
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))

    # Reconstruct the path from end to start
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()

    # If the start node isn't reachable, return an empty path
    return path if path[0] == start else []

In [3]:
def translate_path(path, candidate_node_list):
    new_path = [candidate_node_list[i] for i in path]
    return new_path




In [4]:
def output_small_matrix(X_distance_matrix, candidate_node_list):
    small_matrix = np.zeros((len(candidate_node_list), len(candidate_node_list)))
    for i, ii in enumerate(candidate_node_list):
        for j, jj in enumerate(candidate_node_list):
            small_matrix[i,j] = X_distance_matrix[ii, jj]
    return small_matrix




In [5]:
def cal_candidate_node_list_minimax_path(i, j, minimax_path_matrix, X_distance_matrix):
    
    N = len(X_distance_matrix)
    
    remaining_list = [k for k in np.arange(N) if k != i and k != j]
    candidate_node_list = []
    candidate_node_list.append(i)
    for t in remaining_list:
        if minimax_path_matrix[i,j] < max(minimax_path_matrix[i,t], minimax_path_matrix[t,j]):
            pass
        else:
            candidate_node_list.append(t)
    candidate_node_list.append(j)
 
    return candidate_node_list
        



In [6]:

def cal_path_warm_start_minimax_path(i, j, minimax_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list_minimax_path(i, j, minimax_path_matrix, X_distance_matrix)
    print("candidate_node_list: \n", candidate_node_list)
    print("Size of candidate_node_list: ", len(candidate_node_list))
   
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path_minimax_path(small_matrix, 0, len(candidate_node_list) - 1)
    path = translate_path(path, candidate_node_list)
    
    return path


In [7]:
# APPD stands for all points path distance (APPD).

def check_minimax_path(i,j, X_distance_matrix, APPD_matrix, path):
    temp1= path[:-1]
    temp2= path[1:]
    llll = [X_distance_matrix[temp1[i],temp2[i]] for i in range(len(temp1))]
    ooo = np.max(llll)
    return ooo == APPD_matrix[i,j]

def check_if_temp_path_already_in(all_paths_list, temp_path):
    aa = set(temp_path)
    for bb in all_paths_list:
        bb = set(bb)
        if aa == bb:
            return True
    return False

def cal_all_paths_warm_start_minimax_path(i, j, minimax_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list_minimax_path(i, j, minimax_path_matrix, X_distance_matrix)
 
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path_minimax_path(small_matrix, 0, len(candidate_node_list) - 1)

    all_paths_list = []
    all_paths_list.append(path)

    K = len(candidate_node_list)
    # temp_list = [q for q in range(K) if q not in path]
    
    temp_list = [q for q in range(1, K - 1)]

    for m in temp_list:
        temp_path1 = dijkstra_show_path_minimax_path(small_matrix, 0, m)
        temp_path2 = dijkstra_show_path_minimax_path(small_matrix, m, K - 1)
        temp_path =  temp_path1[:-1] + temp_path2
        if not check_if_temp_path_already_in(all_paths_list, temp_path):
            all_paths_list.append(temp_path)

    all_paths_list = [translate_path(ppp, candidate_node_list) for ppp in all_paths_list]    

    return all_paths_list

In [8]:
# N = 100

# X_distance_matrix = create_distance_matrix(N)


X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")


In [9]:


 
minimax_path_matrix = floyd_warshall_minimax_path(X_distance_matrix)
 

In [10]:

N = len(X_distance_matrix)


In [11]:
i = 10
j = 50

path1 = dijkstra_show_path_minimax_path(X_distance_matrix, i, j)
    
path2 = cal_path_warm_start_minimax_path(i, j, minimax_path_matrix, X_distance_matrix)
print("Paths:")
print(path1)
print(path2)


    

candidate_node_list: 
 [10, 0, 1, 2, 3, 7, 9, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 30, 32, 33, 34, 35, 37, 41, 45, 46, 51, 52, 55, 56, 59, 60, 61, 65, 67, 68, 69, 71, 72, 73, 74, 75, 76, 82, 85, 88, 90, 91, 94, 95, 97, 98, 99, 50]
Size of candidate_node_list:  60
Paths:
[10, 90, 20, 13, 95, 17, 9, 50]
[10, 90, 20, 13, 95, 17, 9, 50]


In [12]:
check_minimax_path(i,j, X_distance_matrix, minimax_path_matrix, path2)



True

In [13]:
all_paths_list = cal_all_paths_warm_start_minimax_path(i, j, minimax_path_matrix, X_distance_matrix)

In [14]:
for path_i in all_paths_list:
    aaa = check_minimax_path(i,j, X_distance_matrix, minimax_path_matrix, path_i)
    print(path_i, aaa)



[10, 90, 20, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 34, 73, 7, 61, 45, 56, 68, 0, 76, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 34, 73, 55, 88, 74, 59, 1, 52, 46, 76, 13, 95, 17, 9, 50] True
[10, 90, 2, 25, 22, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 52, 51, 85, 3, 85, 15, 34, 73, 55, 46, 76, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 34, 73, 7, 61, 82, 88, 74, 20, 13, 95, 17, 9, 50] True
[10, 90, 20, 13, 95, 17, 91, 11, 59, 1, 52, 46, 76, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 52, 46, 76, 12, 85, 15, 34, 25, 22, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 60, 15, 34, 73, 55, 46, 76, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 60, 97, 16, 52, 46, 76, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 60, 37, 18, 94, 41, 15, 34, 73, 55, 46, 76, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 60, 15, 34, 25, 22, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 34, 73, 55, 88, 74, 59, 21, 23, 41, 15, 34, 25, 22, 13, 95, 17, 9, 50] True
[10, 90, 2, 19, 69, 60, 37, 18, 24, 68, 0, 76, 13, 9

In [15]:
len(all_paths_list)

33